In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [1]:
# !wget https://f000.backblazeb2.com/file/malaya-model/v38/summarize/base.pb
# !wget https://f000.backblazeb2.com/file/malaya-model/v38/summarize/small.pb

In [3]:
import tensorflow as tf
from tensorflow.tools.graph_transforms import TransformGraph
from glob import glob
tf.set_random_seed(0)

In [4]:
pbs = glob('*.pb')
pbs

['small.pb', 'base.pb']

In [5]:
import tensorflow_text
import tf_sentencepiece

In [22]:
mapping = {'base.pb': 768 * 2000, 'small.pb': 768 * 2000}

transforms = ['add_default_attributes',
             'remove_nodes(op=Identity, op=CheckNumerics)',
             'fold_batch_norms',
             'fold_old_batch_norms',
             '',
             #'quantize_weights(fallback_min=-10240, fallback_max=10240)',
             'strip_unused_nodes',
             'sort_by_execution_order']

for pb in pbs:
    input_graph_def = tf.GraphDef()
    with tf.gfile.FastGFile(pb, 'rb') as f:
        input_graph_def.ParseFromString(f.read())
        
    t = f'quantize_weights(minimum_size={mapping[pb]})'
    t_cp = transforms[:]
    t_cp[4] = t
    print(t_cp)
        
    print(pb)
    
    transformed_graph_def = TransformGraph(input_graph_def, 
                                           ['inputs'],
                                           ['SentenceTokenizer_1/SentenceTokenizer/SentencepieceDetokenizeOp'], t_cp)
    
    with tf.gfile.GFile(f'{pb}.quantized', 'wb') as f:
        f.write(transformed_graph_def.SerializeToString())

['add_default_attributes', 'remove_nodes(op=Identity, op=CheckNumerics)', 'fold_batch_norms', 'fold_old_batch_norms', 'quantize_weights(minimum_size=1536000)', 'strip_unused_nodes', 'sort_by_execution_order']
small.pb
['add_default_attributes', 'remove_nodes(op=Identity, op=CheckNumerics)', 'fold_batch_norms', 'fold_old_batch_norms', 'quantize_weights(minimum_size=1536000)', 'strip_unused_nodes', 'sort_by_execution_order']
base.pb


In [11]:
def load_graph(frozen_graph_filename, **kwargs):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # https://github.com/onnx/tensorflow-onnx/issues/77#issuecomment-445066091
    # to fix import T5
    for node in graph_def.node:
        if node.op == 'RefSwitch':
            node.op = 'Switch'
            for index in xrange(len(node.input)):
                if 'moving_' in node.input[index]:
                    node.input[index] = node.input[index] + '/read'
        elif node.op == 'AssignSub':
            node.op = 'Sub'
            if 'use_locking' in node.attr:
                del node.attr['use_locking']
        elif node.op == 'AssignAdd':
            node.op = 'Add'
            if 'use_locking' in node.attr:
                del node.attr['use_locking']
        elif node.op == 'Assign':
            node.op = 'Identity'
            if 'use_locking' in node.attr:
                del node.attr['use_locking']
            if 'validate_shape' in node.attr:
                del node.attr['validate_shape']
            if len(node.input) == 2:
                node.input[0] = node.input[1]
                del node.input[1]

    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

In [23]:
g = load_graph('small.pb.quantized')
x = g.get_tensor_by_name('import/inputs:0')
logits = g.get_tensor_by_name('import/SentenceTokenizer_1/SentenceTokenizer/SentencepieceDetokenizeOp:0')

In [24]:
# x, x_len, logits

In [25]:
test_sess = tf.InteractiveSession(graph = g)

In [26]:
x

<tf.Tensor 'import/inputs:0' shape=(?,) dtype=string>

In [27]:
%%time
test_sess.run(logits, feed_dict = {x: ['ringkasan: KUALA LUMPUR: Presiden Perancis Emmanuel Macron tidak menampakkan beliau seorang sosok yang bertamadun, selar Tun Dr Mahathir Mohamad menerusi kemas kini terbaharu di blognya. Bekas Perdana Menteri itu mendakwa, pemerintah tertinggi Perancis itu bersikap primitif kerana menuduh orang Islam terlibat dalam pembunuhan guru yang menghina Islam, malah menegaskan tindakan membunuh bukan ajaran Islam. Jelas Dr Mahathir, sejarah membuktikan bahawa orang Perancis pernah membunuh jutaan manusia, yang ramai mangsanya terdiri dari orang Islam.']})

CPU times: user 14.6 s, sys: 3.31 s, total: 17.9 s
Wall time: 4.59 s


array([b'Presiden Perancis Emmanuel Macron tidak memperlihatkan beliau seorang sosok yang bertamadun, kata Dr. Mahathir'],
      dtype=object)

In [28]:
# %%time
# test_sess.run(logits, feed_dict = {x: [[1,2,3,3,4]], x_len: [[1,1,1,1,1]]})

In [55]:
!rm *.pb*